# 03.07 Merge and Join

## Combining data sets

the main interface for this is pd.merge function, we'll see few examples of how this can work in practice

In [2]:
# Some utility tools
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

pd.merge() implements a number of types of join: 1 to 1, many to 1, and many to many join. 

### one-to-one join

In [2]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
display('df1', 'df2')

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR
,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [4]:
df3= pd.merge(df1, df2)
df3

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


### many-to-many join

In [5]:
df5 = pd.DataFrame({'group': ['Accounting', 'Accounting',
                              'Engineering', 'Engineering', 'HR', 'HR'],
                    'skills': ['math', 'spreadsheets', 'coding', 'linux',
                               'spreadsheets', 'organization']})

In [6]:
pd.merge(df1,df5)

,employee,group,skills
0,Bob,Accounting,math
1,Bob,Accounting,spreadsheets
2,Jake,Engineering,coding
3,Jake,Engineering,linux
4,Lisa,Engineering,coding
5,Lisa,Engineering,linux
6,Sue,HR,spreadsheets
7,Sue,HR,organization


### specification of the merge key

In [7]:
display('df1','df2',"pd.merge(df1,df2, on='employee')")

df1
  employee        group
0      Bob   Accounting
1     Jake  Engineering
2     Lisa  Engineering
3      Sue           HR

df2
  employee  hire_date
0     Lisa       2004
1      Bob       2008
2     Jake       2012
3      Sue       2014

pd.merge(df1,df2, on='employee')
  employee        group  hire_date
0      Bob   Accounting       2008
1     Jake  Engineering       2012
2     Lisa  Engineering       2004
3      Sue           HR       2014

### left_on and right_on

In [8]:
df3 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})

In [10]:
pd.merge(df1, df3, left_on="employee", right_on="name").drop('name', axis=1)

,employee,group,salary
0,Bob,Accounting,70000
1,Jake,Engineering,80000
2,Lisa,Engineering,120000
3,Sue,HR,90000


In [12]:
df6 = pd.DataFrame({'name': ['Peter', 'Paul', 'Mary'],
                    'food': ['fish', 'beans', 'bread']},
                   columns=['name', 'food'])
df7 = pd.DataFrame({'name': ['Mary', 'Joseph'],
                    'drink': ['wine', 'beer']},
                   columns=['name', 'drink'])
display('df6', 'df7', "pd.merge(df6, df7, how='left')")

df6
    name   food
0  Peter   fish
1   Paul  beans
2   Mary  bread

df7
     name drink
0    Mary  wine
1  Joseph  beer

pd.merge(df6, df7, how='left')
    name   food drink
0  Peter   fish   NaN
1   Paul  beans   NaN
2   Mary  bread  wine

In [13]:
#suffix

df8 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [1, 2, 3, 4]})
df9 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'rank': [3, 1, 4, 2]})
pd.merge(df8, df9, on="name", suffixes=["_df8", "_df9"])

,name,rank_df8,rank_df9
0,Bob,1,3
1,Jake,2,1
2,Lisa,3,4
3,Sue,4,2


In [14]:
import os
data_path='C:/Users/miaoxi/Documents/LearnPython/PythonDataScienceHandbook-master/PythonDataScienceHandbook-master/notebooks/data'

pop = pd.read_csv(os.path.join(data_path, 'state-population.csv'))
areas =  pd.read_csv(os.path.join(data_path, 'state-areas.csv'))
abbrevs = pd.read_csv(os.path.join(data_path, 'state-abbrevs.csv'))

In [36]:
display('pop.head()', 'areas.head()', 'abbrevs.head()')

pop.head()
  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

areas.head()
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

abbrevs.head()
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA

In [42]:
#rank US states and territories by their 2010 population density
merged= pop.merge(abbrevs, left_on='state/region', right_on='abbreviation', how='outer')

In [43]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
abbreviation     True
dtype: bool

In [44]:
#figure out which are null
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state,abbreviation
2448,PR,under18,1990,NaN,NaN,NaN
2449,PR,total,1990,NaN,NaN,NaN
2450,PR,total,1991,NaN,NaN,NaN
2451,PR,under18,1991,NaN,NaN,NaN
2452,PR,total,1993,NaN,NaN,NaN


All the null population are from puerto rico prior to the year 2020, data may not be available fromt he original source;
no corresponding entry in the abbrev key, let's figure out which regions lack this match

In [45]:
merged

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL
2,AL,under18,2010,1130966.0,Alabama,AL
3,AL,total,2010,4785570.0,Alabama,AL
4,AL,under18,2011,1125763.0,Alabama,AL
...,...,...,...,...,...,...
2539,USA,total,2010,309326295.0,NaN,NaN
2540,USA,under18,2011,73902222.0,NaN,NaN
2541,USA,total,2011,311582564.0,NaN,NaN
2542,USA,under18,2012,73708179.0,NaN,NaN


In [46]:
merged.loc[merged['state'].isnull(),'state/region'].unique()

array(['PR', 'USA'], dtype=object)

In [47]:
#just two values, we can add quickly
merged.loc[merged['state/region']=='PR','state']='Puerto Rico'
merged.loc[merged['state/region']=='USA','state']='United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
abbreviation     True
dtype: bool

In [48]:
final= pd.merge(merged, areas, left_on='state', right_on='state',how='left')
final.head()

,state/region,ages,year,population,state,abbreviation,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,AL,52423.0
1,AL,total,2012,4817528.0,Alabama,AL,52423.0
2,AL,under18,2010,1130966.0,Alabama,AL,52423.0
3,AL,total,2010,4785570.0,Alabama,AL,52423.0
4,AL,under18,2011,1125763.0,Alabama,AL,52423.0


In [49]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
abbreviation      True
area (sq. mi)     True
dtype: bool

In [50]:
final.loc[final['area (sq. mi)'].isnull(),'state'].unique()

array(['United States'], dtype=object)

In [54]:
final=final.drop('abbreviation',axis=1).dropna()

In [55]:
final.isnull().any()

state/region     False
ages             False
year             False
population       False
state            False
area (sq. mi)    False
dtype: bool

In [58]:
# Now we have all data we need, let's do the calculation we need
# data2010= final[(final['ages']=='total') & (final['year']==2010)]
data2010 = final.query("year ==2010 & ages =='total'")
data2010['density']=data2010['population']/data2010['area (sq. mi)']

C:\Users\miaoxi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [63]:
data2010.sort_values('density',ascending=False).head()

,state/region,ages,year,population,state,area (sq. mi),density
389,DC,total,2010,605125.0,District of Columbia,68.0,8898.897059
2490,PR,total,2010,3721208.0,Puerto Rico,3515.0,1058.665149
1445,NJ,total,2010,8802707.0,New Jersey,8722.0,1009.253268
1914,RI,total,2010,1052669.0,Rhode Island,1545.0,681.339159
293,CT,total,2010,3579210.0,Connecticut,5544.0,645.600649


# 03.08 Aggregation and Grouping

In [1]:
import numpy as np
import pandas as pd

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [6]:
# import seaborn as sns
# planets = sns.load_dataset('planets')

import os
data_path="C:/Users/miaoxi/Documents/LearnPython/seaborn-data-master"
planets= pd.read_csv(os.path.join(data_path,'planets.csv'))

In [14]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [15]:
# inspect missing data
planets.isnull().any()

method            False
number            False
orbital_period     True
mass               True
distance           True
year              False
dtype: bool

In [16]:
#overall look at the planets data
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


## GroupBy: Split, Apply, Combine

### split, apply, combine

    a) the split step is breaking the dataset into groups depending on the value of the specified key
    b) the apply step involves computing some functions, transformation, or filtering within the individual group
    c) the combine step is simply merge the result into one output 

In [3]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [4]:
df.groupby('key')

Notice that what is returned is not df, but DataFrameGroupBy object; And does not actual compuation unitl the aggregation is applied-- "lazy evaluation"

In [5]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


### The GroupBy object

In [17]:
planets.groupby('method')['orbital_period','mass'].median()

,orbital_period,mass
method,,
Astrometry,631.180000,NaN
Eclipse Timing Variations,4343.500000,5.125
Imaging,27500.000000,NaN
Microlensing,3300.000000,NaN
Orbital Brightness Modulation,0.342887,NaN
Pulsar Timing,66.541900,NaN
Pulsation Timing Variations,1170.000000,NaN
Radial Velocity,360.200000,1.260
Transit,5.714932,1.470


In [21]:
# Dispatch methods
planets.groupby('method')['year'].describe()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Astrometry,2.0,2011.500000,2.121320,2010.0,2010.75,2011.5,2012.25,2013.0
Eclipse Timing Variations,9.0,2010.000000,1.414214,2008.0,2009.00,2010.0,2011.00,2012.0
Imaging,38.0,2009.131579,2.781901,2004.0,2008.00,2009.0,2011.00,2013.0
Microlensing,23.0,2009.782609,2.859697,2004.0,2008.00,2010.0,2012.00,2013.0
Orbital Brightness Modulation,3.0,2011.666667,1.154701,2011.0,2011.00,2011.0,2012.00,2013.0
Pulsar Timing,5.0,1998.400000,8.384510,1992.0,1992.00,1994.0,2003.00,2011.0
Pulsation Timing Variations,1.0,2007.000000,NaN,2007.0,2007.00,2007.0,2007.00,2007.0
Radial Velocity,553.0,2007.518987,4.249052,1989.0,2005.00,2009.0,2011.00,2014.0
Transit,397.0,2011.236776,2.077867,2002.0,2010.00,2012.0,2013.00,2014.0


### aggregate, filter, transform, apply

GroupBy objects have aggregate(), filter(), transform() and apply() method that efficiently implement a variety of useful operations before combinging the grouped data

#### aggregation

In [22]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6),
                  'data3':[1,2,3,4,None, 6]},
                   columns = ['key', 'data1', 'data2','data3'])
df

,key,data1,data2,data3
0,A,0,5,1.0
1,B,1,0,2.0
2,C,2,3,3.0
3,A,3,3,4.0
4,B,4,7,NaN
5,C,5,9,6.0


In [23]:
df.groupby('key').aggregate(['min','max'])

data1     data2     data3     
      min max   min max   min  max
key                               
A       0   3     3   5   1.0  4.0
B       1   4     0   7   2.0  2.0
C       2   5     3   9   3.0  6.0

In [25]:
# another useful pattern is to pass a dictionary mapping column 
# names to operations to be applied on that column

df.groupby('key').aggregate({'data1': 'min',
                             'data2':'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


#### filtering

In [26]:
def filter_func(x):
    return x['data2'].std()>4

In [27]:
filter_func(df)

False

In [33]:
df.groupby('key')['data2'].std()>4

key
A    False
B     True
C     True
Name: data2, dtype: bool

In [32]:
df.groupby('key').filter(filter_func)

,key,data1,data2,data3
1,B,1,0,2.0
2,C,2,3,3.0
4,B,4,7,NaN
5,C,5,9,6.0


#### transformation

In [34]:
df.groupby('key').transform(lambda x: x - x.mean())

,data1,data2,data3
0,-1.5,1.0,-1.5
1,-1.5,-3.5,0.0
2,-1.5,-3.0,-1.5
3,1.5,-1.0,1.5
4,1.5,3.5,NaN
5,1.5,3.0,1.5


#### apply

In [41]:
def norm_by_data2(x):
    x['data1']=x['data1']/(x['data2'].sum())
    return x

In [42]:
df.groupby('key').apply(norm_by_data2)

,key,data1,data2,data3
0,A,0.000000,5,1.0
1,B,0.142857,0,2.0
2,C,0.166667,3,3.0
3,A,0.375000,3,4.0
4,B,0.571429,7,NaN
5,C,0.416667,9,6.0


apply() within a groupby is very flexible: the only criterion is that the function takes a dataframe and returns a pandas object or scalar; 

### specifying the split key

In the simple examples before, we split the df on a single column name, this is just one of many options by which the groups can be defined, and we'll go through some other options for group specification here

In [44]:
# The key can be any series or list with a length matching that of the df
# so the more verbose way
df.groupby(df['key']).sum()

,data1,data2,data3
key,,,
A,3,8,5.0
B,5,7,2.0
C,7,12,9.0


In [45]:
# another mehod is to provide a dictionary that maps index to gthe group keys
df2= df.set_index('key')
mapping =  {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}

df2.groupby(mapping).sum()

,data1,data2,data3
consonant,12,19,11.0
vowel,3,8,5.0


In [52]:
#similar to mapping, you can pass any python function that will input the index and output the group
display("df2.groupby('key').mean()","df2.groupby(str).mean()","df2.groupby(str.lower).mean()","df2.groupby([str.lower, mapping]).mean()")

,data1,data2,data3
key,,,
A,1.5,4.0,2.5
B,2.5,3.5,2.0
C,3.5,6.0,4.5
,data1,data2,data3
A,1.5,4.0,2.5
B,2.5,3.5,2.0
C,3.5,6.0,4.5
,data1,data2,data3
a,1.5,4.0,2.5


### Grouping example

In [53]:
# count discovered planets by method and by decade
decade = 10*(planets['year']//10)
decade = decade.astype('str')+'s'

0       2000s
1       2000s
2       2010s
3       2000s
4       2000s
        ...  
1030    2000s
1031    2000s
1032    2000s
1033    2000s
1034    2000s
Name: year, Length: 1035, dtype: object

In [56]:
planets.groupby(['method',decade])['number'].sum().unstack().fillna(0)

year,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0


This shows the power of combining many of the opeartions discussed up to this point when looking at realistic datasets.

# 03.09 Pivot Tables

We could think of pivot tables as essentially a mulitdimensional version of GroupBy aggregation, that is you split-apply-combine, but both the split and the combine happen across not a one dimensional index, but across two dimensional grid

## Motivating Pivot Tables

In [58]:
import numpy as np
import pandas as pd
import os

data_path="C:/Users/miaoxi/Documents/LearnPython/seaborn-data-master"
titanic= pd.read_csv(os.path.join(data_path,'titanic.csv'))

In [59]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


### Pivot Table Syntax

In [60]:
titanic.pivot_table('survived',index='sex', columns='class')

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


### Multi-level pivot tables

In [62]:
age = pd.cut(titanic['age'],[0,18,58,80])

0      (18.0, 58.0]
1      (18.0, 58.0]
2      (18.0, 58.0]
3      (18.0, 58.0]
4      (18.0, 58.0]
           ...     
886    (18.0, 58.0]
887    (18.0, 58.0]
888             NaN
889    (18.0, 58.0]
890    (18.0, 58.0]
Name: age, Length: 891, dtype: category
Categories (3, interval[int64]): [(0, 18] < (18, 58] < (58, 80]]

In [64]:
fare = pd.qcut(titanic['fare'],2)
titanic.pivot_table('survived',['sex',fare],'class',
                    aggfunc='sum',fill_value=0)

class                     First  Second  Third
sex    fare                                   
female (-0.001, 14.454]       0      26     47
       (14.454, 512.329]     91      44     25
male   (-0.001, 14.454]       0       7     32
       (14.454, 512.329]     45      10     15

In [70]:
titanic.pivot_table(index='sex', columns='class', aggfunc={'survived':'sum',
                                           'fare':'mean'}).stack()
#notice that we've omitted the values keyword, when specifying a mapping for aggfunc, this is determined automatically
#but we have to give the index and columns specifically

fare  survived
sex    class                       
female First   106.125798        91
       Second   21.970121        70
       Third    16.118810        72
male   First    67.226127        45
       Second   19.741782        17
       Third    12.661633        47

In [75]:
titanic.pivot_table(index='sex', columns='class', values='survived',margins=True)

#Margins = True means  compute totals along each grouping

class,First,Second,Third,All
sex,,,,
female,0.968085,0.921053,0.500000,0.742038
male,0.368852,0.157407,0.135447,0.188908
All,0.629630,0.472826,0.242363,0.383838
